# Codigo basado en el ejemplo de Dedalus

Se modifico el ejemplo de dedalus **Internally Heated Convection**, el cual resuelve las ecuaciones
\begin{align}
\nabla \cdot \mathbf{u} &= 0\\
\frac{\partial \mathbf{u}}{\partial t} - \nu \nabla^2 \mathbf{u} +\nabla p - \hat{r} T &= -(\mathbf{u} \cdot \nabla)\mathbf{u}\\
\frac{\partial T}{\partial t} - \kappa \nabla^2 T &= -(u\cdot \nabla)T + \kappa T_c
\end{align}
Dado que este ejemplo resuelve ecuaciones similares a las que se quieren resolver, se modifico el codigo para resolver
\begin{align}
\frac{\partial \ln \rho}{\partial t} + \nabla \cdot \mathbf{u} &= -\mathbf{u} \cdot \nabla \ln \rho\\
\frac{\partial \mathbf{u}}{\partial t} - \nu \nabla^2 \mathbf{u} +\frac{\nabla p}{\rho} &= -(\mathbf{u} \cdot \nabla)\mathbf{u}\\
\frac{\partial T}{\partial t}  &= -u\cdot \nabla T 
\end{align}

In [1]:
import sys
import numpy as np
import dedalus.public as d3
import logging
logger = logging.getLogger(__name__)


# Allow restarting via command line
restart = (len(sys.argv) > 1 and sys.argv[1] == '--restart')

# Parameters
Nphi, Ntheta, Nr = 96, 48, 64
Rayleigh = 1e6
Prandtl = 1
dealias = 3/2
stop_sim_time = 20 + 20*restart
timestepper = d3.SBDF2
max_timestep = 0.0002
dtype = np.float64
mesh = None

Gamma = 5/3
sc,Tc,rho_c = 10,200, 11
boundary_s , boundary_rho = 100,0.001
cv=1000

2025-07-03 00:20:15,774 dedalus 0/1 WARNING :: Threading has not been disabled. This may massively degrade Dedalus performance.
2025-07-03 00:20:15,777 dedalus 0/1 WARNING :: We strongly suggest setting the "OMP_NUM_THREADS" environment variable to "1".
2025-07-03 00:20:15,924 numexpr.utils 0/1 INFO :: NumExpr defaulting to 16 threads.


In [2]:
# Bases
coords = d3.SphericalCoordinates('phi', 'theta', 'r')
dist = d3.Distributor(coords, dtype=dtype, mesh=mesh)
ball = d3.BallBasis(coords, shape=(Nphi, Ntheta, Nr), radius=1, dealias=dealias, dtype=dtype)
sphere = ball.surface

# Fields
u = dist.VectorField(coords, name='u',bases=ball)
p = dist.Field(name='p', bases=ball)
s = dist.Field(name='T', bases=ball)
lnrho = dist.Field(bases=ball,name="lnrho")


tau_p = dist.Field(name='tau_p')
tau_u = dist.VectorField(coords, name='tau u', bases=sphere)
tau_s = dist.Field(name='tau T', bases=sphere)
tau_rho = dist.Field(bases=sphere,name="tau_rho")


# Substitutions
phi, theta, r = dist.local_grids(ball)
nu = 1e-1
lift = lambda A: d3.Lift(A, ball, -1)
strain_rate = d3.grad(u) + d3.trans(d3.grad(u))
shear_stress = d3.angular(d3.radial(strain_rate(r=1), index=1))
rho = np.exp(lnrho)
T = Tc*(rho/rho_c)**(Gamma-1)*np.exp((s-sc)/cv)
p = rho*T

lnrho['g'] = np.log(rho_c + (boundary_rho - rho_c) * 0.5 * (1 + np.tanh((r-0.5)/0.2)))
s['g'] = sc + (boundary_s - sc) * 0.5 * (1 + np.tanh((r-0.5)/0.2))
u.fill_random('g')

In [3]:

problem = d3.IVP([lnrho, u, s, tau_u, tau_s], namespace=locals())
problem.add_equation("dt(lnrho) + div(u) = u@grad(lnrho) ")
problem.add_equation("dt(u) - nu*lap(u)  + lift(tau_u) = -grad(p)/rho - u@grad(u)")
problem.add_equation("dt(s) + lift(tau_s) = - u@grad(s)")
problem.add_equation("shear_stress = 0")  # Stress free
problem.add_equation("radial(u(r=1)) = 0")  # No penetration
problem.add_equation("radial(grad(s)(r=1)) = -2")

# Solver
solver = problem.build_solver(timestepper)
solver.stop_sim_time = stop_sim_time

# Initial conditions
if not restart:
    file_handler_mode = 'overwrite'
    initial_timestep = max_timestep
else:
    write, initial_timestep = solver.load_state('checkpoints/checkpoints_s20.h5')
    initial_timestep = 2e-2
    file_handler_mode = 'append'

# Analysis
slices = solver.evaluator.add_file_handler('slices', sim_dt=0.001, max_writes=10, mode=file_handler_mode)
slices.add_task(rho(phi=0), scales=dealias, name='T(phi=0)')
slices.add_task(rho(phi=np.pi), scales=dealias, name='T(phi=pi)')
slices.add_task(rho(phi=3/2*np.pi), scales=dealias, name='T(phi=3/2*pi)')
slices.add_task(rho(r=1), scales=dealias, name='T(r=1)')
checkpoints = solver.evaluator.add_file_handler('checkpoints', sim_dt=1, max_writes=1, mode=file_handler_mode)
checkpoints.add_tasks(solver.state)

# CFL
CFL = d3.CFL(solver, initial_timestep, cadence=10, safety=0.5, threshold=0.1, max_dt=max_timestep)
CFL.add_velocity(u)

# Flow properties
flow = d3.GlobalFlowProperty(solver, cadence=10)
flow.add_property(u@u, name='u2')

# Main loop
try:
    logger.info('Starting main loop')
    while solver.proceed:
        timestep = CFL.compute_timestep()
        solver.step(timestep)
        if (solver.iteration-1) % 1 == 0:
            max_u = np.sqrt(flow.max('u2'))
            logger.info("Iteration=%i, Time=%e, dt=%e, max(u)=%e" %(solver.iteration, solver.sim_time, timestep, max_u))
except:
    logger.error('Exception raised, triggering end of main loop.')
    raise
finally:
    solver.log_stats()

2025-07-03 00:21:02,550 subsystems 0/1 INFO :: Building subproblem matrices 1/47 (~2%) Elapsed: 0s, Remaining: 5s, Rate: 9.8e+00/s
2025-07-03 00:21:03,154 subsystems 0/1 INFO :: Building subproblem matrices 5/47 (~11%) Elapsed: 1s, Remaining: 6s, Rate: 7.1e+00/s
2025-07-03 00:21:03,912 subsystems 0/1 INFO :: Building subproblem matrices 10/47 (~21%) Elapsed: 1s, Remaining: 5s, Rate: 6.8e+00/s
2025-07-03 00:21:04,671 subsystems 0/1 INFO :: Building subproblem matrices 15/47 (~32%) Elapsed: 2s, Remaining: 5s, Rate: 6.7e+00/s
2025-07-03 00:21:05,436 subsystems 0/1 INFO :: Building subproblem matrices 20/47 (~43%) Elapsed: 3s, Remaining: 4s, Rate: 6.7e+00/s
2025-07-03 00:21:06,207 subsystems 0/1 INFO :: Building subproblem matrices 25/47 (~53%) Elapsed: 4s, Remaining: 3s, Rate: 6.6e+00/s
2025-07-03 00:21:06,965 subsystems 0/1 INFO :: Building subproblem matrices 30/47 (~64%) Elapsed: 5s, Remaining: 3s, Rate: 6.6e+00/s
2025-07-03 00:21:07,716 subsystems 0/1 INFO :: Building subproblem matri

/home/lau/miniforge3/envs/dedalus3/lib/python3.13/site-packages/dedalus/core/operators.py:588: RuntimeWarning: overflow encountered in exp
  self.func(arg0.data, out=out.data)
/home/lau/miniforge3/envs/dedalus3/lib/python3.13/site-packages/dedalus/core/arithmetic.py:865: RuntimeWarning: overflow encountered in multiply
  np.multiply(arg0_exp_data, arg1_exp_data, out=out.data)
/home/lau/miniforge3/envs/dedalus3/lib/python3.13/site-packages/dedalus/core/operators.py:384: RuntimeWarning: divide by zero encountered in power
  np.power(arg0.data, arg1, out.data)
/home/lau/miniforge3/envs/dedalus3/lib/python3.13/site-packages/dedalus/core/operators.py:384: RuntimeWarning: overflow encountered in power
  np.power(arg0.data, arg1, out.data)
/home/lau/miniforge3/envs/dedalus3/lib/python3.13/site-packages/dedalus/tools/array.py:117: RuntimeWarning: invalid value encountered in matmul
  temp = np.matmul(matrix, array) # Allocates temp


2025-07-03 00:21:34,159 __main__ 0/1 INFO :: Iteration=7, Time=1.400000e-03, dt=2.000000e-04, max(u)=4.487921e+00
2025-07-03 00:21:35,403 __main__ 0/1 INFO :: Iteration=8, Time=1.600000e-03, dt=2.000000e-04, max(u)=4.487921e+00
2025-07-03 00:21:36,642 __main__ 0/1 INFO :: Iteration=9, Time=1.800000e-03, dt=2.000000e-04, max(u)=4.487921e+00
2025-07-03 00:21:37,478 __main__ 0/1 ERROR :: Exception raised, triggering end of main loop.
2025-07-03 00:21:37,480 solvers 0/1 INFO :: Final iteration: 9
2025-07-03 00:21:37,481 solvers 0/1 INFO :: Final sim time: 0.0018000000000000004
2025-07-03 00:21:37,481 solvers 0/1 INFO :: Setup time (init - iter 0): 59.59 sec
2025-07-03 00:21:37,482 solvers 0/1 INFO :: Timings unavailable because warmup did not complete.


/home/lau/miniforge3/envs/dedalus3/lib/python3.13/site-packages/dedalus/core/evaluator.py:419: SyntaxWarning: invalid escape sequence '\d'
  m = re.match("{}_s(\d+)$".format(base_path.stem), set.stem)
/home/lau/miniforge3/envs/dedalus3/lib/python3.13/site-packages/dedalus/core/transforms.py:162: SyntaxWarning: invalid escape sequence '\s'
  """
/home/lau/miniforge3/envs/dedalus3/lib/python3.13/site-packages/dedalus/core/transforms.py:334: SyntaxWarning: invalid escape sequence '\s'
  """
/home/lau/miniforge3/envs/dedalus3/lib/python3.13/site-packages/dedalus/core/transforms.py:633: SyntaxWarning: invalid escape sequence '\s'
  """


KeyboardInterrupt: 

# Observaciones:

-> El codigo funciona unicamente si se resuelve para la temperatura en vez de para la entropia, supongo que esto es por el termino $$\frac{\nabla p}{\rho} $$dado que este se resuelve en base a sustituciones y por ende, lo que termina haciendo internamente Dedalus es calcular un termino proporcional a $$\frac{\nabla \left ( \rho^{\Gamma} \exp \left ( s \right ) \right )}{\rho}$$.